# 데이터

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split


x1 = np.array([range(100), range(301, 401)]).T                          # ex) 삼전 시가, 고가
x2 = np.array([range(101, 201), range(411, 511), range(150, 250)]).T    # ex) 아모레 시가, 고가, 종가
x3 = np.array([range(100, 200), range(1301, 1401)]).T
y1 = np.array(range(2001, 2101))                                        # 삼전 하루 뒤 종가
y2 = np.array(range(201, 301))                                          # 아모레 하루 뒤 종가

print(x1.shape)
print(x2.shape)
print(x3.shape)
print(y1.shape)
print(y2.shape)

x1_train, x1_test, \
    x2_train, x2_test, \
        x3_train, x3_test, \
            y1_train, y1_test, \
                y2_train, y2_test = train_test_split(x1, x2, x3, y1, y2, train_size=0.7, random_state=3333)

print(x1_train.shape, x1_test.shape,
      x2_train.shape, x2_test.shape,
      x3_train.shape, x3_test.shape,
      y1_train.shape, y1_test.shape,
      y2_train.shape, y2_test.shape)

# x_train = x_train.reshape(-1, 4, 1)
# x_test = x_test.reshape(-1, 4, 1)
# pre_data = pre_data.reshape(-1, 4, 1)

(100, 2)
(100, 3)
(100, 2)
(100,)
(100,)
(70, 2) (30, 2) (70, 3) (30, 3) (70, 2) (30, 2) (70,) (30,) (70,) (30,)


In [2]:
from tensorflow.keras.layers import Dense, Input, concatenate, Concatenate
from tensorflow.keras.models import Model

# 모델 1
input1 = Input(shape=(2, ))
dense1 = Dense(16, activation='relu', name='dense1')(input1) # summary를 봤을 때 보기쉽게 이름 지정
dense2 = Dense(16, activation='relu', name='dense2')(dense1)
dense3 = Dense(16, activation='relu', name='dense3')(dense2)
output1 = Dense(16, activation='relu', name='output1')(dense3)

# 모델 2 
input2 = Input(shape=(3, ))
dense2_1 = Dense(16, activation='linear', name='dense2_1')(input2)
dense2_2 = Dense(16, activation='linear', name='dense2_2')(dense2_1)
output2 = Dense(16, activation='linear', name='output2')(dense2_2)


# 모델 3
input3 = Input(shape=(2, ))
dense3_1 = Dense(16, activation='linear', name='dense3_1')(input3)
dense3_2 = Dense(16, activation='linear', name='dense3_2')(dense3_1)
output3 = Dense(16, activation='linear', name='output3')(dense3_2)
       
# 병합
# merge1 = concatenate([output1, output2, output3], name = 'merge1') # 소문자: 함수
merge1 = Concatenate(name='merge1')([output1, output2, output3])     # 대문자: class
merge2 = Dense(16, activation='relu', name='merge2')(merge1)
merge3 = Dense(16, name='merge3')(merge2) 
merge_output = Dense(1, name = 'merge_output')(merge3)

# 분기 모델 4
dense4_1 = Dense(16, activation='linear', name='dense4_1')(merge_output)
dense4_2 = Dense(16, activation='linear', name='dense4_2')(dense4_1)
output4 = Dense(16, activation='linear', name='output4')(dense4_2)

# 분기 모델 5
dense5_1 = Dense(16, activation='linear', name='dense5_1')(merge_output)
dense5_2 = Dense(16, activation='linear', name='dense5_2')(dense5_1)
output5 = Dense(16, activation='linear', name='output5')(dense5_2)

model = Model(inputs=[input1, input2, input3], outputs=[output4, output5])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 2)]          0           []                               
                                                                                                  
 dense1 (Dense)                 (None, 16)           48          ['input_1[0][0]']                
                                                                                                  
 input_2 (InputLayer)           [(None, 3)]          0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 2)]          0           []                               
                                                                                              

# 컴파일, 훈련

In [3]:
model.compile(loss='mse', optimizer= 'adam')
model.fit([x1_train, x2_train, x3_train], [y1_train, y2_train], epochs=16, batch_size=3)

Epoch 1/16
24/24 [==============================] - 2s 9ms/step - loss: 4207592.0000 - output4_loss: 4137333.5000 - output5_loss: 70257.9609
Epoch 2/16
24/24 [==============================] - 0s 8ms/step - loss: 2666859.5000 - output4_loss: 2619025.5000 - output5_loss: 47833.4375
Epoch 3/16
24/24 [==============================] - 0s 8ms/step - loss: 628756.3125 - output4_loss: 621092.6875 - output5_loss: 7663.5571
Epoch 4/16
24/24 [==============================] - 0s 7ms/step - loss: 92087.4609 - output4_loss: 91112.9609 - output5_loss: 974.5035
Epoch 5/16
24/24 [==============================] - 0s 8ms/step - loss: 8512.8242 - output4_loss: 8108.5508 - output5_loss: 404.2737
Epoch 6/16
24/24 [==============================] - 0s 8ms/step - loss: 2649.7920 - output4_loss: 2232.5635 - output5_loss: 417.2284
Epoch 7/16
24/24 [==============================] - 0s 8ms/step - loss: 2079.1653 - output4_loss: 1691.3456 - output5_loss: 387.8199
Epoch 8/16
24/24 [============================

# 평가

In [4]:
loss = model.evaluate([x1_test, x2_test, x3_test], [y1_test, y2_test])
print("loss(mse): :", loss)

1/1 [==============================] - 0s 200ms/step - loss: 1702.0802 - output4_loss: 1042.6997 - output5_loss: 659.3805
loss(mse): : [1702.0802001953125, 1042.69970703125, 659.3804931640625]
